## CLASSIFICATION MODELS ON TWITTER DATA

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import  accuracy_score
import warnings
warnings.simplefilter('ignore')
classification_result = pd.DataFrame(index=None, columns=['model','train roc_auc','test roc_auc', 'train accuracy', 'test accuracy'])



In [ ]:
df = pd.read_csv('Twitter-Absolute-Sigma-500.data', header = None)


In [202]:
df.columns = ['NCD_0', 'NCD_1', 'NCD_2', 'NCD_3', 'NCD_4', 'NCD_5', 'NCD_6',
'AI_0', 'AI_1', 'AI_2', 'AI_3', 'AI_4', 'AI_5', 'AI_6', 
'AS(NA)_0','AS(NA)_1', 'AS(NA)_2', 'AS(NA)_3', 'AS(NA)_4', 'AS(NA)_5', 'AS(NA)_6', 
'BL_0', 'BL_1', 'BL_2', 'BL_3', 'BL_4', 'BL_5', 'BL_6', 
'NAC_0', 'NAC_1', 'NAC_2', 'NAC_3', 'NAC_4', 'NAC_5', 'NAC_6', 
'AS(NAC)_0', 'AS(NAC)_1', 'AS(NAC)_2', 'AS(NAC)_3', 'AS(NAC)_4', 'AS(NAC)_5', 'AS(NAC)_6', 
'CS_0', 'CS_1', 'CS_2', 'CS_3', 'CS_4', 'CS_5', 'CS_6', 
'AT_0', 'AT_1', 'AT_2', 'AT_3', 'AT_4', 'AT_5', 'AT_6', 
'NA_0', 'NA_1', 'NA_2', 'NA_3', 'NA_4', 'NA_5', 'NA_6',
'ADL_0', 'ADL_1', 'ADL_2', 'ADL_3', 'ADL_4', 'ADL_5', 'ADL_6', 
'NAD_0', 'NAD_1', 'NAD_2', 'NAD_3', 'NAD_4', 'NAD_5', 'NAD_6',
'buzz']

In [203]:
X = df.drop('buzz', axis = 1)
y = df['buzz']

## EXPLORE

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140707 entries, 0 to 140706
Data columns (total 78 columns):
NCD_0        140707 non-null int64
NCD_1        140707 non-null int64
NCD_2        140707 non-null int64
NCD_3        140707 non-null int64
NCD_4        140707 non-null int64
NCD_5        140707 non-null int64
NCD_6        140707 non-null int64
AI_0         140707 non-null int64
AI_1         140707 non-null int64
AI_2         140707 non-null int64
AI_3         140707 non-null int64
AI_4         140707 non-null int64
AI_5         140707 non-null int64
AI_6         140707 non-null int64
AS(NA)_0     140707 non-null float64
AS(NA)_1     140707 non-null float64
AS(NA)_2     140707 non-null float64
AS(NA)_3     140707 non-null float64
AS(NA)_4     140707 non-null float64
AS(NA)_5     140707 non-null float64
AS(NA)_6     140707 non-null float64
BL_0         140707 non-null float64
BL_1         140707 non-null float64
BL_2         140707 non-null float64
BL_3         140707 non-null 

In [146]:
df.describe()

,NCD_0,NCD_1,NCD_2,NCD_3,NCD_4,NCD_5,NCD_6,AI_0,AI_1,AI_2,...,ADL_5,ADL_6,NAD_0,NAD_1,NAD_2,NAD_3,NAD_4,NAD_5,NAD_6,buzz
count,140707.000000,140707.000000,140707.000000,140707.000000,140707.000000,140707.000000,140707.000000,140707.000000,140707.000000,140707.000000,...,140707.000000,140707.000000,140707.000000,140707.000000,140707.000000,140707.000000,140707.000000,140707.000000,140707.000000,140707.000000
mean,172.279823,155.150625,165.464476,176.820549,186.937700,216.209208,243.866510,87.050154,78.639236,84.269574,...,1.113444,1.196131,172.838807,155.630878,165.938674,177.314810,187.463794,216.776294,244.479194,0.197396
std,509.872276,471.573236,495.360236,528.351277,560.331281,632.188378,707.402192,234.731748,218.448179,233.536510,...,1.374287,1.826150,510.937549,472.462733,496.233557,529.286514,561.309487,633.203935,708.436795,0.398035
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,2.000000,3.000000,3.000000,3.000000,4.000000,5.000000,2.000000,2.000000,2.000000,...,1.000000,1.000000,3.000000,2.000000,3.000000,3.000000,3.000000,4.000000,6.000000,0.000000
50%,22.000000,19.000000,20.000000,22.000000,23.000000,28.000000,33.000000,13.000000,11.000000,13.000000,...,1.000000,1.000000,22.000000,19.000000,21.000000,22.000000,23.000000,28.000000,33.000000,0.000000
75%,125.000000,112.000000,119.000000,126.000000,133.000000,161.000000,186.000000,70.000000,64.000000,67.000000,...,1.100000,1.119048,126.000000,113.000000,119.000000,127.000000,134.000000,162.000000,187.000000,0.000000
max,24210.000000,22899.000000,20495.000000,27007.000000,30957.000000,28603.000000,37505.000000,15105.000000,15730.000000,16389.000000,...,185.666672,295.000000,24301.000000,22980.000000,20495.000000,27071.000000,31028.000000,28697.000000,37505.000000,1.000000


## CLASSIFICATION

In [148]:
_, sample_data, _, sample_target = train_test_split(X, y, shuffle = True, test_size = 0.1)

In [149]:
X_train_unscaled, X_test_unscaled, y_train, y_test = train_test_split(sample_data, sample_target, test_size = 0.3, random_state = 101)

In [150]:
MinMax = MinMaxScaler()
X_train = MinMax.fit_transform(X_train_unscaled)
X_test = MinMax.transform(X_test_unscaled)

## KNN CLASSIFICATION

In [151]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

knn = KNeighborsClassifier()
param_grid = {'n_neighbors' : [3,5,7]}
grid_search = GridSearchCV(knn, param_grid, cv = 5, scoring='roc_auc')
grid_search.fit(X_train, y_train)
grid_search.cv_results_['mean_test_score']

array([0.97018671, 0.97784019, 0.98110348])

In [152]:
grid_search.best_params_

{'n_neighbors': 7}

In [153]:
grid_search.best_score_

0.981103478355209

In [154]:
y_knn_predict = grid_search.predict(X_test)
y_knn_train_predict = grid_search.predict(X_train)

In [155]:
from sklearn.metrics import roc_auc_score
print('Train roc_auc_score: %.2f'%roc_auc_score(y_knn_train_predict, y_train))
print('Test roc_auc_score: %.2f '%roc_auc_score(y_knn_predict, y_test))

Train roc_auc_score: 0.97
Test roc_auc_score: 0.95 


In [176]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
predict_test = knn.predict(X_test)
predict_train = knn.predict(X_train)
train_roc_auc = roc_auc_score(y_train, predict_train)
test_roc_auc = roc_auc_score(y_test, predict_test)
train_accuracy = accuracy_score(y_train, predict_train)
test_accuracy = accuracy_score(y_test, predict_test)

classification_result = classification_result.append(pd.Series({'model':'K Neighbors Classifier','train roc_auc':train_roc_auc,'test roc_auc':test_roc_auc, 'train accuracy': train_accuracy, 'test accuracy': test_accuracy}),ignore_index=True)
classification_result

,model,train roc_auc,test roc_auc,train accuracy,test accuracy
0,K Neighbors Classifier,0.955902,0.930112,0.976038,0.962577


## LINEAR SVC

In [157]:
from sklearn.svm import SVC

param_grid = {'C': [0.001, 0.01, 0.1, 0.5, 1, 10, 50, 100, 1000], 'max_iter':[1000,10000] }

grid_search = GridSearchCV(SVC(random_state=0,kernel='linear'), param_grid, cv=5, return_train_score=True)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 0.5, 1, 10, 50, 100, 1000], 'max_iter': [1000, 10000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [158]:
grid_svc_lin.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [160]:
grid_svc_lin.best_params_

{'C': 10}

In [177]:
svc = SVC(C = 50, max_iter = 1000, kernel = 'linear', random_state = 0)
svc.fit(X_train, y_train)
predict_test = svc.predict(X_test)
predict_train = svc.predict(X_train)
train_roc_auc = roc_auc_score(y_train, predict_train)
test_roc_auc = roc_auc_score(y_test, predict_test)
train_accuracy = accuracy_score(y_train, predict_train)
test_accuracy = accuracy_score(y_test, predict_test)

classification_result = classification_result.append(pd.Series({'model':'Linear SVM','train roc_auc':train_roc_auc,'test roc_auc':test_roc_auc, 'train accuracy': train_accuracy, 'test accuracy': test_accuracy}),ignore_index=True)
classification_result

,model,train roc_auc,test roc_auc,train accuracy,test accuracy
0,K Neighbors Classifier,0.955902,0.930112,0.976038,0.962577
1,Linear SVM,0.949074,0.941717,0.970352,0.967551


## LOGISTIC REGRESSION

In [169]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  accuracy_score


In [170]:
logr = LogisticRegression()
param_grid = {'penalty': ['l1', 'l2']}
grid_search = GridSearchCV(logr, param_grid, cv = 5, return_train_score= True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2']}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [171]:
grid_search.cv_results_

{'mean_fit_time': array([0.23674169, 0.11422696]),
 'std_fit_time': array([0.02131809, 0.00539736]),
 'mean_score_time': array([0.00039897, 0.00068097]),
 'std_score_time': array([0.00048864, 0.00036045]),
 'param_penalty': masked_array(data=['l1', 'l2'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'penalty': 'l1'}, {'penalty': 'l2'}],
 'split0_test_score': array([0.96548223, 0.95329949]),
 'split1_test_score': array([0.96294416, 0.94619289]),
 'split2_test_score': array([0.96649746, 0.96294416]),
 'split3_test_score': array([0.97005076, 0.96040609]),
 'split4_test_score': array([0.96597257, 0.95226003]),
 'mean_test_score': array([0.96618946, 0.95502081]),
 'std_test_score': array([0.00228413, 0.0060052 ]),
 'rank_test_score': array([1, 2]),
 'split0_train_score': array([0.96674705, 0.95786267]),
 'split1_train_score': array([0.96738165, 0.95659348]),
 'split2_train_score': array([0.96750857, 0.95494352]),
 'split3_train_score': 

In [172]:
grid_search.best_params_

{'penalty': 'l1'}

In [173]:
grid_search.best_score_

0.9661894608589705

In [178]:
logr = LogisticRegression(penalty = 'l1')
logr.fit(X_train,y_train)
predict_logr = logr.predict(X_test)
predict_testlogr = logr.predict(X_test)
predict_trainlogr = logr.predict(X_train)
train_roc_auc_logr = roc_auc_score(y_train, predict_trainlogr)
test_roc_auc_logr = roc_auc_score(y_test, predict_testlogr)
train_accuracy_logr = accuracy_score(y_train, predict_trainlogr)
test_accuracy_logr = accuracy_score(y_test, predict_testlogr)
classification_result = classification_result.append(pd.Series({'model':'Logistic Regression','train roc_auc':train_roc_auc_logr,'test roc_auc':test_roc_auc_logr, 'train accuracy': train_accuracy_logr, 'test accuracy': test_accuracy_logr}),ignore_index=True)
classification_result

,model,train roc_auc,test roc_auc,train accuracy,test accuracy
0,K Neighbors Classifier,0.955902,0.930112,0.976038,0.962577
1,Linear SVM,0.949074,0.941717,0.970352,0.967551
2,Logistic Regression,0.935908,0.931878,0.967509,0.966840


## POLYNOMIAL SVM


In [180]:
param_grid = {'C': [0.001, 0.01, 0.1, 0.5, 1, 10, 50, 100, 1000], 'max_iter':[1000,10000], 'gamma':[0.001, 0.01, 0.1, 0.5, 1, 10] }

grid_search = GridSearchCV(SVC(random_state=0,kernel='poly'), param_grid, cv=5, return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 0.5, 1, 10, 50, 100, 1000], 'max_iter': [1000, 10000], 'gamma': [0.001, 0.01, 0.1, 0.5, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [181]:
grid_search.cv_results_

{'mean_fit_time': array([1.30484915, 2.24606323, 1.30398078, 2.32230129, 1.3190659 ,
        2.13705764, 1.09989204, 1.10659981, 0.73950267, 0.7370018 ,
        0.44156737, 0.83195906, 1.31972308, 2.13118191, 1.29716063,
        2.30272417, 1.41333323, 2.04666128, 0.74935336, 0.75486765,
        0.56650248, 0.61847482, 0.39743195, 1.18375707, 1.2981627 ,
        2.16059227, 1.29329805, 2.16091013, 1.16125402, 1.16614084,
        0.5516561 , 0.56080117, 0.53008623, 0.5386508 , 0.32455359,
        1.07268634, 1.29075861, 2.13473616, 1.29599738, 2.12784863,
        0.83578839, 0.83052835, 0.53150816, 0.55463467, 0.52045913,
        0.63352003, 0.3176898 , 1.2607964 , 1.31037812, 2.13510461,
        1.31276278, 2.13250313, 0.73739448, 0.74032135, 0.53081245,
        0.54678283, 0.44373879, 0.72052245, 0.31789055, 1.04711046,
        1.30154786, 2.16697221, 1.28392057, 2.15442715, 0.61277156,
        0.61150336, 0.50157175, 0.80153012, 0.32835078, 1.13296056,
        0.32219176, 1.05451975,

In [182]:
grid_search.best_params_

{'C': 1, 'gamma': 0.5, 'max_iter': 1000}

In [183]:
grid_search.best_score_

0.9684231901715911

In [185]:
svc = SVC(C = 1, max_iter = 1000, gamma = 0.5, kernel = 'poly', random_state = 0)
svc.fit(X_train, y_train)
predict_test = svc.predict(X_test)
predict_train = svc.predict(X_train)
train_roc_auc = roc_auc_score(y_train, predict_train)
test_roc_auc = roc_auc_score(y_test, predict_test)
train_accuracy = accuracy_score(y_train, predict_train)
test_accuracy = accuracy_score(y_test, predict_test)

classification_result = classification_result.append(pd.Series({'model':'Polynomial SVM','train roc_auc':train_roc_auc,'test roc_auc':test_roc_auc, 'train accuracy': train_accuracy, 'test accuracy': test_accuracy}),ignore_index=True)
classification_result

,model,train roc_auc,test roc_auc,train accuracy,test accuracy
0,K Neighbors Classifier,0.955902,0.930112,0.976038,0.962577
1,Linear SVM,0.949074,0.941717,0.970352,0.967551
2,Logistic Regression,0.935908,0.931878,0.967509,0.966840
3,Polynomial SVM,0.955124,0.945907,0.970454,0.964946


## RADIAL SVM

In [ ]:
param_grid = {'C':[0.001, 0.01, 0.1, 0.5, 1, 10, 50, 100, 1000], 'max_iter':[1000,10000], 'gamma':[0.001, 0.01, 0.1, 0.5, 1, 10]}

grid_search = GridSearchCV(SVC(random_state=0,kernel='rbf'), param_grid, cv=5, return_train_score=True)
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.cv_results_

In [112]:
grid_search.best_params_

{'C': 100, 'gamma': 0.1, 'max_iter': 1000}

In [113]:
grid_search.best_score_

0.9552123552123553

In [114]:
svc = SVC(C = 100, max_iter = 1000, gamma = 0.1, kernel = 'rbf', random_state = 0)
svc.fit(X_train, y_train)
predict_test = svc.predict(X_test)
predict_train = svc.predict(X_train)
train_roc_auc = roc_auc_score(y_train, predict_train)
test_roc_auc = roc_auc_score(y_test, predict_test)
train_accuracy = accuracy_score(y_train, predict_train)
test_accuracy = accuracy_score(y_test, predict_test)

classification_results = classification_results.append(pd.Series({'model':'Radial SVM','train roc_auc':train_roc_auc,'test roc_auc':test_roc_auc, 'train accuracy': train_accuracy, 'test accuracy': test_accuracy}),ignore_index=True)
classification_results

,model,train roc_auc,test roc_auc,train accuracy,test accuracy
0,K Neighbors Classifier,0.921906,0.922898,0.959073,0.955036
1,Linear SVM,0.929995,0.940442,0.962934,0.962230
2,Logistic Regression,0.890555,0.886818,0.947490,0.944245
3,Polynomial SVM,0.933797,0.928415,0.964479,0.958633
4,Radial SVM,0.931448,0.916389,0.965251,0.955036


## DECISION TREE CLASSIFIER

In [190]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=10)
param_grid = {'max_depth': [5, 10, 20, 50, 100]}

grid_search = GridSearchCV(dtc, param_grid, cv = 5, return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=10,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [5, 10, 20, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [191]:
grid_search.cv_results_

{'mean_fit_time': array([2.68828535, 5.19343076, 7.5341146 , 8.4768208 , 8.46947641]),
 'std_fit_time': array([0.09499156, 0.17755106, 0.62829591, 0.29382737, 0.27973008]),
 'mean_score_time': array([0.01419725, 0.01514297, 0.01504579, 0.01560054, 0.01438198]),
 'std_score_time': array([1.80943653e-03, 1.65362148e-03, 2.38841890e-03, 2.34309066e-05,
        3.09234038e-03]),
 'param_max_depth': masked_array(data=[5, 10, 20, 50, 100],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 5},
  {'max_depth': 10},
  {'max_depth': 20},
  {'max_depth': 50},
  {'max_depth': 100}],
 'split0_test_score': array([0.96502538, 0.9613198 , 0.94928934, 0.94771574, 0.94771574]),
 'split1_test_score': array([0.96542972, 0.96060714, 0.9512158 , 0.94730697, 0.94730697]),
 'split2_test_score': array([0.96563277, 0.96147013, 0.95162191, 0.95101274, 0.95101274]),
 'split3_test_score': array([0.96283887, 0.96024977, 0.95106102, 

In [192]:
grid_search.best_params_

{'max_depth': 5}

In [194]:
grid_search.best_score_

0.9645866753304769

In [196]:
dtc = DecisionTreeClassifier(max_depth= 5, random_state= 10)
dtc.fit(X_train, y_train)
predict_test = dtc.predict(X_test)
predict_train = dtc.predict(X_train)
train_roc_auc = roc_auc_score(y_train, predict_train)
test_roc_auc = roc_auc_score(y_test, predict_test)
train_accuracy = accuracy_score(y_train, predict_train)
test_accuracy = accuracy_score(y_test, predict_test)

classification_result = classification_result.append(pd.Series({'model':'Decision Tree Classifier','train roc_auc':train_roc_auc,'test roc_auc':test_roc_auc, 'train accuracy': train_accuracy, 'test accuracy': test_accuracy}),ignore_index=True)
classification_result

,model,train roc_auc,test roc_auc,train accuracy,test accuracy
0,K Neighbors Classifier,0.955902,0.930112,0.976038,0.962577
1,Linear SVM,0.949074,0.941717,0.970352,0.967551
2,Logistic Regression,0.935908,0.931878,0.967509,0.966840
3,Polynomial SVM,0.955124,0.945907,0.970454,0.964946
4,Decision Tree Classifier,0.940172,0.934661,0.966902,0.964016


## BEST MODEL

## Based on the classification results, we see that Linear SVM has the highest roc_auc, accuracy score for test sets.

## LINEAR SVM 

In [204]:
X_train_unscaled, X_test_unscaled, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)

In [205]:
MinMax = MinMaxScaler(feature_range= (0,1))
X_train = MinMax.fit_transform(X_train_unscaled)
X_test = MinMax.transform(X_test_unscaled)

In [ ]:
param_grid = {'C': [0.001, 0.01, 0.1, 0.5, 1, 10, 50, 100, 1000], 'max_iter':[1000,10000] }
grid_search = GridSearchCV(SVC(random_state=0,kernel='linear'), param_grid, cv=5, return_train_score=True)
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.cv_results_

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
svc = SVC(C = 100, max_iter = 10000, kernel = 'linear', random_state = 0)
svc.fit(X_train, y_train)
predict_test = svc.predict(X_test)
predict_train = svc.predict(X_train)
print('Train ROC_AUC : %.2f%%'%(roc_auc_score(y_train, predict_train)*100))
print('Test ROC_AUC : %.2f%%'%(roc_auc_score(y_test, predict_test)*100))
print('Train accuracy : %.2f%%'%(accuracy_score(y_train, predict_train)*100))
print('Test accuracy : %.2f%%'%(accuracy_score(y_test, predict_test)*100))
